# SOCI 415 Network Analysis - CBDB Dataset

The China Biographical Database Abstract: The China Biographical
Database is a freely accessible relational database with biographical
information about approximately 641,568 individuals as of August 2024,
currently mainly from the 7th through 19th centuries. With both online
and offline versions, the data is meant to be useful for statistical,
social network, and spatial analysis as well as serving as a kind of
biographical reference. The image below shows the spatial distribution
of a cross dynastic subset of 190,000 people in CBDB by basic
affiliations

Display values within the dataset

In [1]:
import sqlite3
import pandas as pd

db_path = r'/arc/project/st-lknelson-1/alexr951/git/data/latest.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# List all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:", tables)

conn.close()

Tables in database: [('ADDR_BELONGS_DATA',), ('ADDR_CODES',), ('ADDR_PLACE_DATA',), ('ADDR_XY',), ('ADDRESSES',), ('ALTNAME_CODES',), ('ALTNAME_DATA',), ('APPOINTMENT_CODE_TYPE_REL',), ('APPOINTMENT_CODES',), ('APPOINTMENT_TYPES',), ('ASSOC_CODE_TYPE_REL',), ('ASSOC_CODES',), ('ASSOC_DATA',), ('ASSOC_TYPES',), ('ASSUME_OFFICE_CODES',), ('BIOG_ADDR_CODES',), ('BIOG_ADDR_DATA',), ('BIOG_INST_CODES',), ('BIOG_INST_DATA',), ('BIOG_MAIN',), ('BIOG_SOURCE_DATA',), ('BIOG_TEXT_DATA',), ('CHORONYM_CODES',), ('Copy Of CopyTables',), ('CopyMissingTables',), ('CopyTables',), ('CopyTablesDefault',), ('COUNTRY_CODES',), ('DATABASE_LINK_CODES',), ('DATABASE_LINK_DATA',), ('DYNASTIES',), ('ENTRY_CODE_TYPE_REL',), ('ENTRY_CODES',), ('ENTRY_DATA',), ('ENTRY_TYPES',), ('ETHNICITY_TRIBE_CODES',), ('EVENT_CODES',), ('EVENTS_ADDR',), ('EVENTS_DATA',), ('EXTANT_CODES',), ('ForeignKeys',), ('FormLabels',), ('GANZHI_CODES',), ('HOUSEHOLD_STATUS_CODES',), ('INDEXYEAR_TYPE_CODES',), ('KIN_DATA',), ('KIN_Mournin

Load the one we want

In [2]:
db_path = r'/arc/project/st-lknelson-1/alexr951/git/data/latest.db'

# Connect to the database
conn = sqlite3.connect(db_path)

# Replace 'person' with the actual table name you want to load
df = pd.read_sql_query("SELECT * FROM KIN_DATA", conn)

# Show the first few rows
print(df.head())

conn.close()

   c_personid  c_kin_id  c_kin_code  c_source        c_pages  \
0           0    689344         337   64847.0  088  周真夫人黃氏墓誌   
1           1         2         180       0.0           None   
2           1     13310         210    7596.0           3047   
3           1     13311         180       0.0           None   
4           1    119997         182    2229.0           None   

                  c_notes c_autogen_notes c_created_by c_created_date  \
0                    None            None   Qi Xinghai       20250329   
1  [待考。《宋史·安惇傳》云，惇二子郊、邦。]            None          TTS       20070312   
2                    None            None          TTS       20070312   
3                    None            None          TTS       20070312   
4      據宋史列傳CBDB宋史分傳#2159            None         load       20121116   

  c_modified_by c_modified_date  
0    Qi Xinghai        20250329  
1          None            None  
2          None            None  
3          None            None  
4     

Build the NetworkX Graph

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add edges with kinship type as edge attribute
for _, row in df.iterrows():
    person = row['c_personid']
    kin = row['c_kin_id']
    kin_type = row['c_kin_code']
    G.add_edge(person, kin, kinship=kin_type)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 278262
Number of edges: 272717

Visualize the network

In [5]:
import sqlite3
import pandas as pd

# -------- Update this only once for all cells ------- #
DB_PATH = r'/arc/project/st-lknelson-1/alexr951/git/data/latest.db'
# --------------------------------------------------- #

def load_kin_data(db_path=DB_PATH):
    """Return a DataFrame with all rows from KIN_DATA."""
    with sqlite3.connect(db_path) as conn:
        df = pd.read_sql_query("SELECT * FROM KIN_DATA", conn)
    # Drop obvious self-loops
    df = df[df["c_personid"] != df["c_kin_id"]]
    return df

kin_df = load_kin_data()
print("Rows:", len(kin_df))
kin_df.head()


def basic_overview(df):
    print("=== BASIC COLUMN SUMMARY ===")
    print(df.dtypes)                 # data types
    print("\nNull counts:")
    print(df.isna().sum())
    print("\nUnique counts:")
    for col in df.columns:
        print(f"  {col}: {df[col].nunique():,}")

basic_overview(kin_df)


Rows: 543809
=== BASIC COLUMN SUMMARY ===
c_personid           int64
c_kin_id             int64
c_kin_code           int64
c_source           float64
c_pages             object
c_notes             object
c_autogen_notes     object
c_created_by        object
c_created_date      object
c_modified_by       object
c_modified_date     object
dtype: object

Null counts:
c_personid              0
c_kin_id                0
c_kin_code              0
c_source               14
c_pages            139587
c_notes            417131
c_autogen_notes    510431
c_created_by            1
c_created_date          2
c_modified_by      517420
c_modified_date    517420
dtype: int64

Unique counts:
  c_personid: 277,411
  c_kin_id: 277,976
  c_kin_code: 426
  c_source: 454
  c_pages: 24,017
  c_notes: 37,534
  c_autogen_notes: 13,960
  c_created_by: 168
  c_created_date: 1,990
  c_modified_by: 150
  c_modified_date: 1,154

In [6]:
from collections import Counter

def relationship_and_people_stats(df):
    total_rels = len(df)
    unique_people = set(df["c_personid"]) | set(df["c_kin_id"])
    kin_type_counts = Counter(df["c_kin_code"])

    print("=== RELATIONSHIP & PEOPLE STATS ===")
    print(f"Total relationships  : {total_rels:,}")
    print(f"Unique individuals   : {len(unique_people):,}")
    print("\nTop 10 kinship codes:")
    for kin, cnt in kin_type_counts.most_common(10):
        share = cnt / total_rels * 100
        print(f"  {kin:<10} {cnt:>8,}  ({share:5.1f}%)")

relationship_and_people_stats(kin_df)


=== RELATIONSHIP & PEOPLE STATS ===
Total relationships  : 543,809
Unique individuals   : 278,257

Top 10 kinship codes:
  180          88,917  ( 16.4%)
  75           88,636  ( 16.3%)
  126          56,512  ( 10.4%)
  125          56,487  ( 10.4%)
  134          28,301  (  5.2%)
  62           28,101  (  5.2%)
  243          27,533  (  5.1%)
  135          24,464  (  4.5%)
  48           19,910  (  3.7%)
  255          19,468  (  3.6%)

In [8]:
import networkx as nx
import numpy as np

def quick_network_snapshot(df):
    G = nx.from_pandas_edgelist(
        df,
        source="c_personid",
        target="c_kin_id",
        edge_attr="c_kin_code",
        create_using=nx.Graph(),
    )

    degrees = [d for _, d in G.degree()]
    comps   = [len(c) for c in nx.connected_components(G)]

    print("=== NETWORK SNAPSHOT ===")
    print(f"Nodes                 : {G.number_of_nodes():,}")
    print(f"Edges                 : {G.number_of_edges():,}")
    print(f"Density               : {nx.density(G):.8f}")
    print(f"Connected components  : {len(comps):,}")
    print(f"Largest component     : {max(comps):,} nodes")
    print("\nDegree distribution:")
    print(f"  mean   {np.mean(degrees):.2f}")
    print(f"  median {np.median(degrees):.2f}")
    print(f"  max    {np.max(degrees)}")
    print(f"  std    {np.std(degrees):.2f}")

quick_network_snapshot(kin_df)


=== NETWORK SNAPSHOT ===
Nodes                 : 278,257
Edges                 : 272,691
Density               : 0.00000704
Connected components  : 30,357
Largest component     : 52,992 nodes

Degree distribution:
  mean   1.96
  median 1.00
  max    147
  std    2.57

In [9]:
import sqlite3
import pandas as pd

#DB_PATH = r'/arc/project/st-lknelson-1/alexr951/git/data/latest.db'

def decode_kinship_relationships():
    """Get the actual meaning of your kinship codes"""
    with sqlite3.connect(DB_PATH) as conn:
        # Load kinship codes lookup table
        kinship_lookup = pd.read_sql_query("SELECT * FROM KINSHIP_CODES", conn)
        
        # Load your KIN_DATA
        kin_data = pd.read_sql_query("SELECT * FROM KIN_DATA", conn)
        kin_data = kin_data[kin_data['c_personid'] != kin_data['c_kin_id']]
        
    print(f"=== RELATIONSHIP TYPES IN YOUR DATA ===")
    print(f"Total kinship records: {len(kin_data):,}")
    
    # Get the actual relationship meanings
    relationships = kin_data['c_kin_code'].value_counts().head(15)
    print(f"\nTop 15 relationship types (by frequency):")
    for code, count in relationships.items():
        pct = count/len(kin_data)*100
        print(f"  Code {code}: {count:,} ({pct:.1f}%)")
    
    print(f"\nKinship lookup table structure:")
    print(kinship_lookup.columns.tolist())
    print(f"Lookup table has {len(kinship_lookup)} relationship definitions")
    
    return kin_data, kinship_lookup

kin_df, lookup_df = decode_kinship_relationships()


=== RELATIONSHIP TYPES IN YOUR DATA ===
Total kinship records: 543,809

Top 15 relationship types (by frequency):
  Code 180: 88,917 (16.4%)
  Code 75: 88,636 (16.3%)
  Code 126: 56,512 (10.4%)
  Code 125: 56,487 (10.4%)
  Code 134: 28,301 (5.2%)
  Code 62: 28,101 (5.2%)
  Code 243: 27,533 (5.1%)
  Code 135: 24,464 (4.5%)
  Code 48: 19,910 (3.7%)
  Code 255: 19,468 (3.6%)
  Code 111: 18,410 (3.4%)
  Code 76: 7,826 (1.4%)
  Code 181: 7,076 (1.3%)
  Code 176: 4,986 (0.9%)
  Code 182: 4,638 (0.9%)

Kinship lookup table structure:
['c_kincode', 'c_kin_pair1', 'c_kin_pair2', 'c_kin_pair_notes', 'c_kinrel_chn', 'c_kinrel', 'c_kinrel_alt', 'c_pick_sorting', 'c_upstep', 'c_dwnstep', 'c_marstep', 'c_colstep', 'c_kinrel_simplified']
Lookup table has 479 relationship definitions

In [10]:
import networkx as nx
import numpy as np

def analyze_family_network_structure(df):
    """Understand the shape and clustering of your kinship network"""
    
    print("=== NETWORK STRUCTURE & FAMILY PATTERNS ===")
    
    # Build the network
    G = nx.from_pandas_edgelist(df, source='c_personid', target='c_kin_id', 
                               create_using=nx.Graph())
    
    print(f"Network size: {G.number_of_nodes():,} people, {G.number_of_edges():,} relationships")
    print(f"Network density: {nx.density(G):.8f}")
    
    # Family cluster analysis (connected components = family groups)
    components = list(nx.connected_components(G))
    family_sizes = sorted([len(comp) for comp in components], reverse=True)
    
    print(f"\n=== FAMILY CLUSTER ANALYSIS ===")
    print(f"Total family groups: {len(components):,}")
    print(f"Largest family: {family_sizes[0]:,} people")
    print(f"Top 10 family sizes: {family_sizes[:10]}")
    print(f"Families with 100+ people: {sum(1 for size in family_sizes if size >= 100)}")
    print(f"Single-person 'families': {sum(1 for size in family_sizes if size == 1)}")
    
    # Individual connectivity patterns  
    degrees = dict(G.degree())
    degree_values = list(degrees.values())
    
    print(f"\n=== INDIVIDUAL CONNECTIVITY ===")
    print(f"Average relationships per person: {np.mean(degree_values):.2f}")
    print(f"Most connected person has: {max(degree_values)} relationships")
    print(f"People with only 1 relationship: {sum(1 for d in degree_values if d == 1):,}")
    print(f"People with 10+ relationships: {sum(1 for d in degree_values if d >= 10):,}")
    
    return G, components, degrees

G, families, connectivity = analyze_family_network_structure(kin_df)


=== NETWORK STRUCTURE & FAMILY PATTERNS ===
Network size: 278,257 people, 272,691 relationships
Network density: 0.00000704

=== FAMILY CLUSTER ANALYSIS ===
Total family groups: 30,357
Largest family: 52,992 people
Top 10 family sizes: [52992, 357, 331, 294, 267, 236, 188, 160, 152, 130]
Families with 100+ people: 16
Single-person 'families': 0

=== INDIVIDUAL CONNECTIVITY ===
Average relationships per person: 1.96
Most connected person has: 147 relationships
People with only 1 relationship: 205,427
People with 10+ relationships: 7,755

## Posibilities of what we can do

Louvain (ML) - Can be applied to sub divide the 30,357 connected
components (families) Family Tree Hierarchical Structure: - Look at how
over time families evolved Degree centrality for important family
members (Some have 147 edges from them) Betweenness Centrality for
Bridges/Connectors between large families (Maybe a women who was married
off etc)  
- Look at how networks change across generations and time periods
Spatial Network Analysis - Look at how people and networks change over
time Cross-Dynasty Comparisons - compare family structures across
different Chinese dynasties (depending on how far back the data goes)